In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import os

In [45]:
df = pd.read_csv('quora-question-pairs/train.csv')

In [46]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [47]:
test_data = pd.read_csv('quora-question-pairs/test.csv')

In [48]:
test_data.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [49]:
X_train = df.iloc[:,:5].values
Y_train = df.iloc[:,5:].values

In [50]:
X_testq1 = test_data.iloc[:400001,1:2].values
X_testq2 = test_data.iloc[:400001, 2:].values

In [51]:
s1 = X_train[:,3]
s2 = X_train[:,4]

In [52]:
def tokenize(s):
    tokens = []
    tokens = [word_tokenize(str(sentence)) for sentence in s]

    rm1 = []
    for w in tokens:
        sm = re.sub('[^A-Za-z]',' ', str(w))
        x = re.split("\s", sm)
        rm1.append(x)
        
    return rm1

In [53]:
def lower_case(s):
    #Removing whitespaces    
    for sent in s:
        while '' in sent:
            sent.remove('')

    # Lowercasing
    low = []
    for i in s:
        i = [x.lower() for x in i]
        low.append(i)
        
    return low
    

In [54]:
def lemmatize(s):
    lemma = []
    wnl = WordNetLemmatizer()
    for doc in s:
        tokens = [wnl.lemmatize(w) for w in doc]
        lemma.append(tokens)

    # Removing Stopwords
    filter_words = []
    Stopwords = set(stopwords.words('english'))

    #ab = spell('nd')
    for sent in lemma:
        tokens = [w for w in sent if w not in Stopwords]
        filter_words.append(tokens)

    space = ' ' 
    sentences = []
    for sentence in filter_words:
        sentences.append(space.join(sentence))
        
    return sentences

In [55]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [56]:
X_train_q1 = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
X_train_q1 = pad_sequences(X_train_q1, maxlen = 30, padding='post')

# X_train_q2 = tokenizer.texts_to_sequences(np.array(listq2))
X_train_q2 = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
X_train_q2 = pad_sequences(X_train_q2, maxlen = 30, padding='post')

In [57]:
X_test_q1 = tokenizer.texts_to_sequences(X_testq1.ravel())
X_test_q1 = pad_sequences(X_test_q1,maxlen = 30, padding='post')

X_test_q2 = tokenizer.texts_to_sequences(X_testq2.astype(str).ravel())
X_test_q2 = pad_sequences(X_test_q2, maxlen = 30, padding='post')

In [58]:
word_index = tokenizer.word_index

In [59]:
embedding_index = {}
with open('glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [60]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [61]:
# Model for Q1
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
model_q1 = tf.keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q1.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q1.add(Dropout(0.2))
model_q1.add(LSTM(128, return_sequences = True))
model_q1.add(LSTM(128))
model_q1.add(Dense(60, activation = 'tanh'))
model_q1.add(Dense(2, activation = 'sigmoid'))

In [62]:
# Model for Q2
model_q2 = tf.keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q2.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q2.add(Dropout(0.2))
model_q2.add(LSTM(128, return_sequences = True))
model_q2.add(LSTM(128))
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(2, activation = 'sigmoid'))

In [63]:
# Merging the output of the two models,i.e, model_q1 and model_q2
mergedOut = Multiply()([model_q1.output, model_q2.output])

mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(2, activation = 'sigmoid')(mergedOut)

In [64]:
new_model = Model([model_q1.input, model_q2.input], mergedOut)
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
history = new_model.fit([X_train_q1,X_train_q2],Y_train, batch_size = 2000, epochs = 10)

Epoch 1/10
203/203 [==============================] - 108s 451ms/step - loss: 0.6528 - accuracy: 0.6296
Epoch 2/10
203/203 [==============================] - 90s 442ms/step - loss: 0.5592 - accuracy: 0.7113
Epoch 3/10
203/203 [==============================] - 91s 450ms/step - loss: 0.5116 - accuracy: 0.7495
Epoch 4/10
203/203 [==============================] - 91s 449ms/step - loss: 0.4802 - accuracy: 0.7667
Epoch 5/10
203/203 [==============================] - 91s 449ms/step - loss: 0.4550 - accuracy: 0.7793
Epoch 6/10
203/203 [==============================] - 91s 449ms/step - loss: 0.4297 - accuracy: 0.7918
Epoch 7/10
203/203 [==============================] - 92s 453ms/step - loss: 0.4088 - accuracy: 0.8023
Epoch 8/10
203/203 [==============================] - 91s 448ms/step - loss: 0.3886 - accuracy: 0.8125
Epoch 9/10
203/203 [==============================] - 91s 447ms/step - loss: 0.3705 - accuracy: 0.8219
Epoch 10/10
203/203 [==============================] - 91s 449ms/step - 

In [65]:
y_pred = new_model.predict([X_test_q1, X_test_q2], batch_size=2000, verbose=1)
y_pred += new_model.predict([X_test_q1, X_test_q2], batch_size=2000, verbose=1)
y_pred /= 2

201/201 [==============================] - 25s 125ms/step
